<a href="https://colab.research.google.com/github/annabellesauve/Empathy-Detection/blob/main/Pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**bold text**
# Configurations







In [2]:
import os
import pandas as pd
import tqdm
import torch
import math

In [3]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
#Set the path to the data folder, datafile and output folder and files
root_folder = '/content/drive/My Drive'
data_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/empathy_dataset'))
model_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/pre-training model'))
output_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/pre-tarining output'))
tokenizer_folder = os.path.abspath(os.path.join(root_folder, 'Empathy Detection Project/tokenizer'))

test_filename='interpretations_Test-reddit.csv'
datafile= 'interpretations-reddit.csv'
outputfile = 'interpretationsafterMLM.csv'

datafile_path = os.path.abspath(os.path.join(data_folder,datafile))
testfile_path = os.path.abspath(os.path.join(data_folder,test_filename))
outputfile_path = os.path.abspath(os.path.join(output_folder,outputfile))

# Load Training and Testing Datasets

In [5]:
# Load the train dataset
train_df=pd.read_csv(datafile_path, header=0, usecols=[2,3])
# Show the count of rows
print('Num Examples: ',len(train_df))
print('Null Values\n', train_df.isna().sum())
print(train_df)
# Drop rows with Null values 
train_df.dropna(inplace=True)
print('Num Examples: ',len(train_df))

Num Examples:  3084
Null Values
 seeker_post      0
response_post    0
dtype: int64
                                            seeker_post                                      response_post
0         Help. Help me. I dunno what I'm doing anymore  That's pretty vague, do you not know what you'...
1     I'm done saying I love you to her because I do...  idk what a Red pill means exactly but my advic...
2     Always feel like I'm being criticized and mock...  I think it's social anxiety , that creates par...
3     My diet becomes fucked when i get depressed.. ...  By any chance do you think you're in a loop. J...
4     I hate not knowing why. I was diagnosed with d...  depression. not sadness which is caused by som...
...                                                 ...                                                ...
3079  does anyone else keep forgetting stuff the nee...  All day, every day. It's definitely not just y...
3080  What does depression feel like?. Honest questi...  lik

In [6]:
# Load the test dataset 
test_df=pd.read_csv(testfile_path, header=0)
print('Num Examples: ',len(test_df))
print('Null Values\n', test_df.isna().sum())
print(test_df)
# there are no null values

Num Examples:  1001
Null Values
 seeker_post      0
response_post    0
dtype: int64
                                            seeker_post                                      response_post
0         Help. Help me. I dunno what I'm doing anymore  That's pretty vague, do you not know what you'...
1     I'm done saying I love you to her because I do...  idk what a Red pill means exactly but my advic...
2     Always feel like I'm being criticized and mock...  I think it's social anxiety , that creates par...
3     My diet becomes fucked when i get depressed.. ...  By any chance do you think you're in a loop. J...
4     I hate not knowing why. I was diagnosed with d...  depression. not sadness which is caused by som...
...                                                 ...                                                ...
996   There is literally nothing anyone can say to m...  I'm not going to lie to you and say there's a ...
997   Circle of my life. I'm depressed and fat, so I...  Sam

# Create the dataset to train a tokenizer

In [7]:


# Drop the files from the output dir
txt_files_dir = "./text_split"
!rm -rf {txt_files_dir}
!mkdir {txt_files_dir}



In [8]:
# Store values in a dataframe column (Series object) to files, one file per record
def column_to_files(column, prefix, txt_files_dir):
    # The prefix is a unique ID to avoid to overwrite a text file
    i=prefix
    #For every value in the df, with just one column
    for row in column.to_list():
      # Create the filename using the prefix ID
      file_name = os.path.join(txt_files_dir, str(i)+'.txt')
      try:
        # Create the file and write the column text to it
        f = open(file_name, 'wb')
        f.write(row.encode('utf-8'))
        f.close()
      except Exception as e:  #catch exceptions(for eg. empty rows)
        print(row, e) 
      i+=1
    # Return the last ID
    return i

In [9]:
data = train_df["response_post"]
# Removing the end of line character \n
data = data.replace("\n"," ")
# Set the ID to 0
prefix=0
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
# Print the last ID
print(prefix)

3084


In [10]:
data = test_df["response_post"]
# Removing the end of line character \n
data = data.replace("\n"," ")
print(len(data))
# Create a file for every description value
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

1001
4085


In [11]:
data = train_df["seeker_post"]
data = data.replace("\n"," ")
print(len(data))
prefix = column_to_files(data, prefix, txt_files_dir)
print(prefix)

3084
7169


# Train Tokenizer

In [12]:
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1
import tensorflow as tf 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-xt48pdr8
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-xt48pdr8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 61 kB 402 kB/s 
     |████████████████████████████████| 596 kB 12.1 MB/s 
     |████████████████████████████████| 895 kB 31.1 MB/s 
     |████████████████████████████████| 3.3 MB 37.4 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3254938 sha256=b45e899a58656d1cbee5f7dac71a5c6a881ee6232a519bbcbe92cbcf943504a4
  Stored in directory: /tmp/pip-ephem-wheel-cache-aue7w2qr/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uni

In [13]:
paths = [str(x) for x in Path(".").glob("text_split/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True)

# Customize training
tokenizer.train(files=paths, vocab_size=8192, min_frequency=2,
                show_progress=True,
                special_tokens=[
                                "<s>",
                                "<pad>",
                                "</s>",
                                "<unk>",
                                "<mask>",
])

In [14]:
tokenizer

Tokenizer(vocabulary_size=8192, model=ByteLevelBPE, add_prefix_space=False, lowercase=True, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [15]:
# Tokenizer. save_model () is outdated so we used .save_pretrained()
tokenizer.save_model(tokenizer_folder)

['/content/drive/My Drive/Empathy Detection Project/tokenizer/vocab.json',
 '/content/drive/My Drive/Empathy Detection Project/tokenizer/merges.txt']

In [16]:
# Create the tokenizer using vocab.json and mrege.txt files
tokenizer = ByteLevelBPETokenizer(
    os.path.abspath(os.path.join(tokenizer_folder,'vocab.json')),
    os.path.abspath(os.path.join(tokenizer_folder,'merges.txt'))
)

In [17]:
# Prepare the tokenizer
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [18]:
tokenizer.encode("That's pretty vague, do you not know what you're doing in regards to a specific section of your life? Like school or work?")

Encoding(num_tokens=35, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [19]:
tokenizer.encode("That's pretty vague, do you not know what you're doing in regards to a specific section of your life? Like school or work?").tokens

['<s>',
 'T',
 'hat',
 "'s",
 'Ġpretty',
 'Ġvague',
 ',',
 'Ġdo',
 'Ġyou',
 'Ġnot',
 'Ġknow',
 'Ġwhat',
 'Ġyou',
 "'re",
 'Ġdoing',
 'Ġin',
 'Ġregards',
 'Ġto',
 'Ġa',
 'Ġspecific',
 'Ġse',
 'ction',
 'Ġof',
 'Ġyour',
 'Ġlife',
 '?',
 'Ġ',
 'L',
 'i',
 'ke',
 'Ġschool',
 'Ġor',
 'Ġwork',
 '?',
 '</s>']

# Train Language Model


In [20]:
TRAIN_BATCH_SIZE = 8   # input batch size for training (default: 64)
VALID_BATCH_SIZE = 8    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 10       # number of epochs to train (default: 10)
LEARNING_RATE = 1e-4    # learning rate (default: 0.001)
WEIGHT_DECAY = 0.01
SEED = 12               # random seed (default: 42)
MAX_LEN = 128
SUMMARY_LEN = 7

In [21]:
!nvidia-smi

Sat Dec  4 21:06:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    32W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# A) Define the model

In [22]:
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()
print(gpu_available)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


In [23]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=8192,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [24]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  49816064


In [25]:
from transformers import RobertaTokenizerFast
# Create the tokenizer from a trained one
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_folder, max_len=MAX_LEN)

In [26]:
tokenizer

PreTrainedTokenizerFast(name_or_path='/content/drive/My Drive/Empathy Detection Project/tokenizer', vocab_size=8192, model_max_len=128, is_fast=True, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

# B) Building the training Dataset

In [27]:
from torch.utils.data.dataset import Dataset

numeric_feature_names = ['seeker_post', 'response_post']
numeric_features = train_df[numeric_feature_names]
dataset = tf.data.experimental.make_csv_dataset(
  datafile_path,
  batch_size=8,
  select_columns = ["seeker_post","response_post"])

class CustomDataset(Dataset):
    def __init__(self, df, tokenizer):
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []
        
        for example in df.values:
            x=tokenizer.encode_plus(example, max_length = MAX_LEN, truncation=True, padding=True)
            self.examples += [x.input_ids]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i])



In [28]:
# Create the train and evaluation dataset
train_dataset = CustomDataset(train_df['response_post'], tokenizer)
eval_dataset = CustomDataset(test_df['response_post'], tokenizer)

# C) Define the Data Collactor to Mask Training Data 

---



In [29]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)


# D) Initialize and Run Trainer

In [30]:
from transformers import Trainer, TrainingArguments

print(model_folder)
# Define the training arguments
training_args = TrainingArguments(
    output_dir=model_folder,
    overwrite_output_dir=True,
    evaluation_strategy = 'epoch',
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=VALID_BATCH_SIZE,
    save_steps=8192,
    #eval_steps=4096,
    save_total_limit=1,
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #prediction_loss_only=True,
)
trainer.train()

***** Running training *****
  Num examples = 3084
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3860


/content/drive/My Drive/Empathy Detection Project/pre-training model


Epoch,Training Loss,Validation Loss
1,No log,6.291363
2,6.429400,5.943820
3,5.992400,5.810211
4,5.767700,5.658578
5,5.767700,5.660955
6,5.696400,5.610998
7,5.617500,5.529716
8,5.539300,5.466363
9,5.539300,5.391001
10,5.494000,5.456916


***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8
***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3860, training_loss=5.760010809231298, metrics={'train_runtime': 946.0339, 'train_samples_per_second': 32.599, 'train_steps_per_second': 4.08, 'total_flos': 920466171586560.0, 'train_loss': 5.760010809231298, 'epoch': 10.0})

In [31]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 1001
  Batch size = 8


Perplexity: 233.14


In [32]:
trainer.save_model(model_folder)

Saving model checkpoint to /content/drive/My Drive/Empathy Detection Project/pre-training model
Configuration saved in /content/drive/My Drive/Empathy Detection Project/pre-training model/config.json
Model weights saved in /content/drive/My Drive/Empathy Detection Project/pre-training model/pytorch_model.bin


# Test Trained Model Using a Pipeline

In [33]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model_folder,
    tokenizer=tokenizer_folder
)


loading configuration file /content/drive/My Drive/Empathy Detection Project/pre-training model/config.json
Model config RobertaConfig {
  "_name_or_path": "/content/drive/My Drive/Empathy Detection Project/pre-training model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 8192
}

loading configuration file /content/drive/My Drive/Empathy Detection Project/pre-training model/config.json
Model config RobertaConfi

In [34]:
# Test masked language model
fill_mask("Yeah, this <mask> is a strong one. Felt it for a lot of my childhood and  adulthood. The truth is that I never dealt with it constructively, or at least in a way that could help me get to the root of that feeling. Part of the reason I think you and I are on this in the first place is because we were never taught how to deal with that feeling constructively. In my case, I used constant denial and self-numbing ... At some point, I just assumed the pit was a part of me I could choose to ignore. You seem to be actively trying to experience it. You are engaging your sadness, which is objectively good for working through depression. It may not be what you might call 'conventional' but everyone does their thing a little differently :) If what you are doing no longer works for you though, it might be best to start looking for other (healthy) ways of working through that discomfort. Personally, when I feel that pit, I know what thoughts tend to follow it. Fortunately, my therapists (and my stay at hospital) helped me come up with ways to attack that pit when it hit me hard. In order to redirect those thoughts that create it, I try to reach out to anyone in my support net. It's hard because I feel like they won't give a shit, but most of the time, they do (but I often choose to ignore that reality). Or I will simply write some positive words here, on this. Reaching out to others here in positive ways is not just for others, but for my own sake as well. As you put so well, giving that pit a 'purpose'. But don't cover that feeling up. Continue to engage with it, as best as you can. It's there for a reason. Hopefully, the more you work with it, the easier it is to process and let go.")

Token indices sequence length is longer than the specified maximum sequence length for this model (448 > 128). Running this sequence through the model will result in indexing errors


[{'score': 0.0658024474978447,
  'sequence': "Yeah, this, is a strong one. Felt it for a lot of my childhood and  adulthood. The truth is that I never dealt with it constructively, or at least in a way that could help me get to the root of that feeling. Part of the reason I think you and I are on this in the first place is because we were never taught how to deal with that feeling constructively. In my case, I used constant denial and self-numbing... At some point, I just assumed the pit was a part of me I could choose to ignore. You seem to be actively trying to experience it. You are engaging your sadness, which is objectively good for working through depression. It may not be what you might call 'conventional' but everyone does their thing a little differently :) If what you are doing no longer works for you though, it might be best to start looking for other (healthy) ways of working through that discomfort. Personally, when I feel that pit, I know what thoughts tend to follow it. 